In [1]:
### imports

# external modules
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import importlib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
sys.path.append('../src/classifiers')
sys.path.append('../src/cloudfitters')
import HistStruct
importlib.reload(HistStruct)
import SubHistStruct
importlib.reload(SubHistStruct)
import DataLoader
importlib.reload(DataLoader)
import AutoEncoder
importlib.reload(AutoEncoder)
import SeminormalFitter
import GaussianKdeFitter
import HyperRectangleFitter
importlib.reload(SeminormalFitter)
importlib.reload(GaussianKdeFitter)
importlib.reload(HyperRectangleFitter)

2022-06-21 22:39:21.341165: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.1-8d929/x86_64-centos7-gcc8-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.1-f3599/x86_64-centos7-gcc8-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_forest:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/python/framework:/cvmfs/sft.cern.ch/lcg/releases/java/8u222-884d8/x86_64-centos7-gcc8-opt/jre/lib/amd64:/cvmfs/sft.

<module 'HyperRectangleFitter' from '/eos/home-i01/k/khowey/SWAN_projects/ML4DQMDC-PixelAE/KH-AutoencoderTest/../src/cloudfitters/HyperRectangleFitter.py'>

In [2]:
### Speed Controls and Run Mode

# Disables all plots for large datasets where speed is more important
createPlots = False

# Control for the notebook - turn off user-friendly mode to enable faster runtimes
userfriendly = True

In [3]:
### Evaluation Parameters

# Select the bias towards recall against precision, treated as a factor (so < 1 biases towards precision, 1 is equal importance, and > 1 biases towards recall)
biasFactor = 20

In [4]:
### Data Controls and Selection - 1D Autoncoder

# The directory data is located in
datadir = '../data/'

# Select a list of good runs to train on in development training_mode
# Should be validated by eye
trainrunsls = {'2017B':{
#                    "297057":[[-1]], 
#                    "297099":[[-1]], 
#                    "297101":[[-1]],
#                    "297113":[[-1]], 
#                    "297114":[[-1]], 
                    "297175":[[-1]],   # A decently clean histogram
#                    "297177":[[-1]],
#                    "297179":[[-1]], 
#                    "297215":[[-1]],
#                    "297218":[[-1]],
#                    "297225":[[-1]],
#                    "297296":[[-1]], 
#                    "297411":[[-1]],
#                    "297426":[[-1]],  
#                    "297431":[[-1]],
#                    "297434":[[-1]], 
#                    "297468":[[-1]],
#                    "297483":[[-1]],
#                    "297486":[[-1]],
#                    "297503":[[-1]],
#                    "297557":[[-1]],
#                   "297598":[[-1]],
#                   "297604":[[-1]],   # A decently clean histogram
                   "297620":[[-1]],   # A decently clean histogram
                   "297659":[[-1]],   # An okay histogram
                   "297670":[[-1]],   # A decently clean histogram
#                    "297674":[[-1]],
#                    "297678":[[-1]],   # A particularly messy histogram
#                    "297722":[[-1]],   # A decently clean histogram
#                    "298997":[[-1]],
#                    "299061":[[-1]],
                   "299065":[[-1]],   # A decently clean histogram
                   "299067":[[-1]],   # A decently clean histogram
                   "299096":[[-1]],
                   "299149":[[-1]],
#                    "299178":[[-1]],   # A decently clean histogram
#                    "299184":[[-1]],   # A particularly messy histogram
#                    "299185":[[-1]],   # A decently clean histogram
#                    "299327":[[-1]],
#                    "299329":[[-1]], 
#                    "299480":[[-1]]    # A decently clean histogram
                    },
               '2017C':{
                   "299369":[[-1]]
               },
              '2018':{ # needs to be re-checked, not guaranteed to be fully correct or representative.   
                  "315267":[[-1]] 
              }
}

# Select a list of good runs to test on in development training_mode
# Should be validated by eye
goodrunsls = {'2017B':{
#                    "297057":[[-1]], 
#                    "297099":[[-1]], 
#                    "297101":[[-1]],
#                    "297113":[[-1]], 
#                    "297114":[[-1]], 
                    "297175":[[-1]],   # A decently clean histogram
#                    "297177":[[-1]],
#                    "297179":[[-1]], 
#                    "297215":[[-1]],
#                    "297218":[[-1]],
#                    "297225":[[-1]],
#                    "297296":[[-1]], 
#                    "297411":[[-1]],
#                    "297426":[[-1]],  
#                    "297431":[[-1]],
#                    "297434":[[-1]], 
#                    "297468":[[-1]],
#                    "297483":[[-1]],
#                    "297486":[[-1]],
#                    "297503":[[-1]],
#                    "297557":[[-1]],
#                    "297598":[[-1]],
#                    "297604":[[-1]],   # A decently clean histogram
#                    "297620":[[-1]],   # A decently clean histogram
                    "297659":[[-1]],   # An okay histogram
                    "297670":[[-1]],   # A decently clean histogram
                    "297674":[[-1]],
#                    "297678":[[-1]],   # A particularly messy histogram
                    "297722":[[-1]],   # A decently clean histogram
#                    "298997":[[-1]],
#                    "299061":[[-1]],
                    "299065":[[-1]],   # A decently clean histogram
                    "299067":[[-1]],   # A decently clean histogram
#                    "299096":[[-1]],
#                    "299149":[[-1]],
#                    "299178":[[-1]],   # A decently clean histogram
#                    "299184":[[-1]],   # A particularly messy histogram
                    "299185":[[-1]],   # A decently clean histogram
                    "299327":[[-1]],
#                    "299329":[[-1]], 
                    "299480":[[-1]]    # A decently clean histogram
                    },
              '2017C':{
                  "299368":[[-1]]
              },
              '2018':{ # needs to be re-checked, not guaranteed to be fully correct or representative.   
                  "315267":[[-1]] 
              }
}


badrunsls = {'2017B':
                {
                    #"297048":[[-1]],
                    #"297282":[[-1]],
                    #"297283":[[-1]],
                    #"297284":[[-1]],
                    #"297287":[[-1]],
                    #"297288":[[-1]],
                    #"297289":[[-1]],
                    "299316":[[-1]],
                    "299317":[[-1]],
                    "299318":[[-1]],
                    "299324":[[-1]],
                    "299326":[[-1]],
                    #"301086":[[88,126]],
                    #"301086":[[89,89]],
                    #"303948":[[1710,1710]],
                    "297047":[[-1]], #close but, true bad for all 8
                    "297169":[[-1]], #true bad for all 8
#                   "297211":[[-1]], #Reconstructs well
#                   "299325":[[-1]], #Reconstructs well
                    "297664":[[-1]], #true bad for all 8
                    "299317":[[-1]], #true bad for all 8
                    "297169":[[-1]], #true bad for all 8
#                   "297502":[[-1]]
                },
             '2017C':{
                 "300781":[[-1]], # bad for tracking (pixels were excluded.
                 "300079":[[-1]], # is bad for strips and then also for tracking
                 "302029":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good 
                 "300576":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good
                 "300574":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good
                 "300282":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good
                 "301912":[[-1]], # Half bad for pixels (lost HV or readout card)  
                 "301086":[[-1]], # Half bad for pixels (lost HV or readout card)  
                 "300283":[[-1]], # Half bad for pixels (lost HV or readout card) 
                 "300282":[[-1]], # Half bad for pixels (lost HV or readout card) 
                 "300281":[[-1]], # Half bad for pixels (lost HV or readout card) 
                 "300239":[[-1]], # Half bad for pixels (lost HV or readout card)
                 "301394":[[-1]], # Marginal for pixels
                 "301183":[[-1]], # Marginal for pixels
                 "300398":[[-1]], # Marginal for pixels
                 "300389":[[-1]], # Marginal for pixels
                 "300365":[[-1]]  # Marginal for pixels
              },
             '2017E':{
                 "304740":[[-1]], # Bad for pixels and tracking - holes in PXLayer 1
                 "304776":[[-1]], # Bad for pixels and tracking - holes in PXLayer 1
                 "304506":[[-1]], # Portcard problem for pixels
                 "304507":[[-1]], # Portcard problem for pixels 
                 "303989":[[-1]], # Bad for pixels, power supply died
                 "303824":[[-1]]  # Partly bad for strips due to a test
             },
             '2017F':{
                 "306422":[[-1]], # Partly bad for strips - 2 data readouts failed 
                 "306423":[[-1]], # Partly bad for strips - 2 data readouts failed
                 "306425":[[-1]], # Partly bad for strips - 2 data readouts failed
                 "305440":[[-1]], # Partly bad for strips - 1 data readout failed
                 "305441":[[-1]], # Partly bad for strips - 1 data readout failed
                 "305249":[[-1]], # Bad for pixels - half of disk failed 
                 "305250":[[-1]], # Bad for pixels - half of disk failed
                 "305064":[[-1]], # Marginal for pixels - some readout failed
             },
            '2018': # needs to be re-checked, not guaranteed to be fully correct or representative.
                {
                #"317479":[[-1]],
                "317480":[[-1]],
                "317481":[[-1]],
                "317482":[[-1]],
                #"319847":[[1,35]]
            }}


# The year and era being used
year = '2017'
era = 'B'

# Create a list of histograms to include
# Pair histograms to be combined on the same line
histnames = [
    ['NormalizedHitResiduals_TIB__Layer__1', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__1', 'NormalizedHitResiduals_TIB__Layer__2', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__2',
     'NormalizedHitResiduals_TIB__Layer__3', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__3' , 'NormalizedHitResiduals_TIB__Layer__4', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__4'],
    ['chargeInner_PXLayer_1', 'chargeOuter_PXLayer_1', 'adc_PXLayer_1'],
    ['chargeInner_PXLayer_2', 'chargeOuter_PXLayer_2', 'adc_PXLayer_2'],
    ['chargeInner_PXLayer_3', 'chargeOuter_PXLayer_3', 'adc_PXLayer_3'],
    ['chargeInner_PXLayer_4', 'chargeOuter_PXLayer_4', 'adc_PXLayer_4'],
    ['charge_PXDisk_+1', 'adc_PXDisk_+1'],
    ['charge_PXDisk_-1', 'adc_PXDisk_-1'],
    ['charge_PXDisk_+2', 'adc_PXDisk_+2'],
    ['charge_PXDisk_-2', 'adc_PXDisk_-2'],
    ['charge_PXDisk_+3', 'adc_PXDisk_+3'],
    ['charge_PXDisk_-3', 'adc_PXDisk_-3'],
    ['NormalizedHitResiduals_TOB__Layer__1', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__1', 'NormalizedHitResiduals_TOB__Layer__2', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__2',
     'NormalizedHitResiduals_TOB__Layer__3', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__3' , 'NormalizedHitResiduals_TOB__Layer__4', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__4']
]

# Read new data or use previously saved data & should data be saved
readnew = True
save = False

In [5]:
### Define Run Properties - Combined Autoencoder
# in this cell all major run properties are going to be set,

# Set whether to train globally or locally or in a development/testing mode
training_mode = 'development'

# Selects whether to create a new histstruct or use a saved one
readnew = True

# Select whether to save a new histstruct
save = False

In [6]:
### Model Controls and Selection - 1D Autoencoder

plotNames = 'Test'
name = plotNames+'plots'

# Choose whether to train a new model or load one
trainnew = True
savemodel = True
modelname = plotNames

# Evaluate models seperately, as an ensemble, both, or neither
individualEval = True
ensembleEval = False

In [7]:
### Define Training Mode Parameters - Combined Autoencoder
if training_mode == 'global':
    runsls_training = None # use none to not add a mask for training (can e.g. use DCS-bit on mask)
    runsls_good = None # use none to not add a mask for good runs (can e.g. use averages of training set)
    runsls_bad = badrunsls[year] # predefined bad runs
    print('selected runs/lumisections for training: all')
    
elif training_mode == 'local':
    # train locally on a small set of runs
    # - either on n runs preceding a chosen application run,
    # - or on the run associated as reference to the chosen application run.
    # - this only works for a single era
    
    available_runs = dfu.get_runs( dfu.select_dcson( csvu.read_csv('../data/DF'+year+era+'_'+histnames[0][0]+'.csv') ) )
    # Cherry picked really bad run
    run_application = 299316
    #run_application = 299317
    run_application_index = available_runs.index(run_application)
    # select training set
    usereference = False
    if usereference:
        run_reference = rru.get_reference_run( run_application, jsonfile='../utils/json_allRunsRefRuns.json' )
        if run_reference<0:
            raise Exception('no valid reference run has been defined for run {}'.format(run_application))
        runsls_training = jsonu.tuplelist_to_jsondict([(run_reference,[-1])])
    else:
        ntraining = 5
        offset = 0 # normal case: offset = 0 (just use 5 previous runs)
        
        # Selects the 5 previous runs for training
        runsls_training = jsonu.tuplelist_to_jsondict([(el,[-1]) for el in available_runs[run_application_index-ntraining-offset:run_application_index-offset]])
    #runsls_bad = badrunsls[year]
    #runsls_good = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    runsls_bad = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    runsls_good = runsls_training
    print('selected runs/lumisections for training: ')
    print(runsls_training)
    print('selected runs/lumisections as good test set:')
    print(runsls_good)
    print('selected runs/lumisections as bad test set:')
    print(runsls_bad)
        
elif training_mode == 'development':
    # train on a user-defined subset of runs
    
    # Select runs to be used in training from the user-defined list
    runsls_training = trainrunsls[year + era]
    # Select bad runs to test on in the user-defined list
    runsls_bad = badrunsls[year + era]
    # Select good runs to test on in the user-defined list
    runsls_good = goodrunsls[year + era]
    
    print('selected runs/lumisections for training: ')
    print(runsls_training)
    print('selected runs/lumisections as good test set:')
    print(runsls_good)
    print('selected runs/lumisections as bad test set:')
    print(runsls_bad)

selected runs/lumisections for training: 
{'297175': [[-1]], '297620': [[-1]], '297659': [[-1]], '297670': [[-1]], '299065': [[-1]], '299067': [[-1]], '299096': [[-1]], '299149': [[-1]]}
selected runs/lumisections as good test set:
{'297175': [[-1]], '297659': [[-1]], '297670': [[-1]], '297674': [[-1]], '297722': [[-1]], '299065': [[-1]], '299067': [[-1]], '299185': [[-1]], '299327': [[-1]], '299480': [[-1]]}
selected runs/lumisections as bad test set:
{'299316': [[-1]], '299317': [[-1]], '299318': [[-1]], '299324': [[-1]], '299326': [[-1]], '297047': [[-1]], '297169': [[-1]], '297664': [[-1]]}


In [8]:
### Data Import

# Create a new HistStruct from the data
if readnew:
    # Initializations
    dloader = DataLoader.DataLoader()
    histstruct = SubHistStruct.SubHistStruct()
    histstruct.reset_histlist(histnames)
    
    # Unpack histnames and add every histogram individually
    for histnamegroup in histnames:
        for histname in histnamegroup:
            print('Adding {}...'.format(histname))
            
            # Bring the histograms into memory from storage for later use
            filename = datadir + year + era + '/DF' + year + era + '_' + histname + '.csv'
            df = dloader.get_dataframe_from_file( filename )
            
            # In case of local training, we can remove most of the histograms
            if( runsls_training is not None and runsls_good is not None and runsls_bad is not None ):
                runsls_total = {k: v for d in (runsls_training, runsls_good, runsls_bad) for k, v in d.items()}
                df = dfu.select_runsls( df, runsls_total )
                
            # Store the data in the histstruct object managing this whole thing
            histstruct.add_dataframe( df, rebinningfactor = 3 )
        
    print('Found {} histograms'.format(len(histstruct.runnbs)))

# Load a previously saved HistStruct
else:
    # Load histstruct from storage
    histstruct = HistStruct.HistStruct.load('test.pk1')
    
    # Count of bad runs, presumably for later use
    nbadruns = len([name for name in list(histstruct.masks.keys()) if 'bad' in name])
    
if userfriendly:
    print('Created a histstruct with the following properties:')
    print('- number of histogram types: {}'.format(len(histstruct.histnames)))
    print('- number of lumisections: {}'.format(len(histstruct.lsnbs)))

Adding NormalizedHitResiduals_TIB__Layer__1...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__1...
Adding NormalizedHitResiduals_TIB__Layer__2...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__2...
Adding NormalizedHitResiduals_TIB__Layer__3...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__3...
Adding NormalizedHitResiduals_TIB__Layer__4...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__4...
Adding chargeInner_PXLayer_1...
Adding chargeOuter_PXLayer_1...
Adding adc_PXLayer_1...
WARNING in hist_utils.py / rebinhists: no rebinning performed since no suitable reduction factor was given. The rebinning factor (3) is not a divisor of the number of bins (34)
Adding chargeInner_PXLayer_2...
Adding chargeOuter_PXLayer_2...
Adding adc_PXLayer_2...
WARNING in hist_utils.py / rebinhists: no rebinning performed since no suitable reduction factor was given. The rebinning factor (3) is not a divisor of the number of bins (34)
Adding chargeInner_PXLayer_3...
Adding chargeOuter

In [9]:
### Add Masks to Data

if readnew:
    histstruct.add_dcsonjson_mask( 'dcson' )
    histstruct.add_goldenjson_mask('golden' )
    histstruct.add_highstat_mask( 'highstat' )
    histstruct.add_stat_mask( 'lowstat', max_entries_to_bins_ratio=100 )
    if runsls_training is not None: histstruct.add_json_mask( 'training', runsls_training )
    if runsls_good is not None: histstruct.add_json_mask( 'good', runsls_good )
        
    # Distinguishing bad runs
    nbadruns = 0
    if runsls_bad is not None:
        print(runsls_bad)
        histstruct.add_json_mask( 'bad', runsls_bad )
        
        # Special case for bad runs: add a mask per run (different bad runs have different characteristics)
        nbadruns = len(runsls_bad.keys())
        for i,badrun in enumerate(runsls_bad.keys()):
            histstruct.add_json_mask( 'bad{}'.format(i), {badrun:runsls_bad[badrun]} )
            
    if save:
        histstruct.save('test.pk1')
if userfriendly: print('Assigned masks: {}'.format(list(histstruct.masks.keys())))

{'299316': [[-1]], '299317': [[-1]], '299318': [[-1]], '299324': [[-1]], '299326': [[-1]], '297047': [[-1]], '297169': [[-1]], '297664': [[-1]]}
Assigned masks: ['dcson', 'golden', 'highstat', 'lowstat', 'training', 'good', 'bad', 'bad0', 'bad1', 'bad2', 'bad3', 'bad4', 'bad5', 'bad6', 'bad7']


In [10]:
### Plotting the input data for analysis

if((training_mode=='local' or training_mode == 'development') and createPlots):

    # training and application runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','training'],['dcson','highstat','good']],
                                labellist = ['training','testing'],
                                colorlist = ['blue','green']
                              )
    
    # application run and bad test runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['dcson','highstat','bad']],
                                labellist = ['good','bad'],
                                colorlist = ['green','red']
                              )
    
elif( training_mode=='global' and createPlots):
    
    # bad test runs
    for i in [0,1,2,3,4,5,6]:
        histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['dcson','highstat','bad{}'.format(i)]],
                                labellist = ['typical good histograms','bad'],
                                colorlist = ['blue','red'],
                                transparencylist = [0.01,1.]
                                    )

In [11]:
def define_concatamash_autoencoder(histstruct):
    
    histslist = []
    vallist = []
    autoencoders = []
    if trainnew:
        for i,histnamegroup in enumerate(histnames):
            
            train_normhist = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'training']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
            X_train, X_val = train_test_split(train_normhist, test_size=0.4, random_state=42)
            
            if userfriendly:
                print('\nNow Defining model {}/'.format(i + 1) 
                      + str(len(histnames)))
                print(' - Size of training set: {}'.format(X_train.shape))
            
            ## Model parameters
            print(X_train.shape)
            
            input_dim = X_train.shape[2] #num of predictor variables
            Input_layers=[Input(shape=input_dim) for i in range((X_train.shape[1]))]
            
            # Defining layers
            conc_layer = Concatenate()(Input_layers)
            encoder = Dense(256, activation="tanh")(conc_layer)
            encoder = Dense(128, activation='relu')(encoder)
            
            encoder = Dense(32, activation='relu')(encoder)
            
            decoder = Dense(128, activation="relu")(encoder)
            decoder = Dense(256, activation="tanh")(decoder)
            
            Output_layers=[Dense(input_dim, activation="tanh")(decoder) for i in range(X_train.shape[1])]

            autoencoder = Model(inputs=Input_layers, outputs=Output_layers)
            autoencoder.summary()
            autoencoders.append(autoencoder)
            
            histslist.append(X_train)
            vallist.append(X_val)
     
    # Return the histograms stored 2-Dimensionally and the autoencoders corresponding
    return(histslist, vallist, autoencoders, train_normhist)

In [12]:
(histslist, vallist, autoencoders, train_normhist) = define_concatamash_autoencoder(histstruct)


Now Defining model 1/12
 - Size of training set: (1006, 8, 34)
(1006, 8, 34)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 34)]         0                                            
________________

2022-06-21 22:39:41.677392: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.1-8d929/x86_64-centos7-gcc8-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.1-f3599/x86_64-centos7-gcc8-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_forest:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/python/framework:/cvmfs/sft.cern.ch/lcg/releases/java/8u222-884d8/x86_64-centos7-gcc8-opt/jre/lib/amd64:/cvmfs/sft.cern.ch/lc

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 102)          0           input_9[0][0]                    
                                                                 input_10[0][0]             

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 102)          0           input_18[0][0]                   
                                                                 input_19[0][0]             

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
concatenate_7 (Concatenate)     (None, 68)           0           input_25[0][0]                   
                                                                 input_26[0][0]                   
__________________________________________________________________________________________________
dense_59 (Dense)                (None, 256)          17664       concatenate_7[0][0]        


Now Defining model 12/12
 - Size of training set: (1006, 8, 34)
(1006, 8, 34)
Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_34 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_35 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_36 (InputLayer)           [(None, 34)]         0                                            
____________

In [13]:
### Trains a combined autoencoder for every merge set
def train_concatamash_autoencoder(histstruct, histslist, vallist, autoencoders):
    
    # Iterate through the training data to train corresponding autoencoders
    for i in range(len(histslist)):
        
        if userfriendly: print('\nNow training model {}/'.format(i + 1) + str(len(histslist)))
        
        # Set variables to temporary values for better transparency
        X_train = histslist[i]
        X_val = vallist[i]
        autoencoder = autoencoders[i]
        
        
        ## Model parameters
        nb_epoch = 200
        batch_size = 50
        
        #checkpoint_filepath = 'checkpoint'
        #model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        #   filepath=checkpoint_filepath,
        #   save_weights_only=False,
        #   verbose=1,
        #   save_best_only=True,
        #   monitor='val_loss',
        #   mode='min')
        
        # Tell the model when to stop
        earlystop = EarlyStopping(monitor='val_loss',
            min_delta=1e-7,
            patience=20,
            verbose=1,
            mode='auto',
            baseline=None,
            restore_best_weights=True,
        )
        lr =0.001
        opt = keras.optimizers.Adam(learning_rate=lr)
        
        autoencoder.compile(loss='mse',
                            optimizer=opt)
        
        ## Train autoencoder
        train = autoencoder.fit(x=[X_train[:,i] for i in range(X_train.shape[1])],
                                y=[X_train[:,i] for i in range(X_train.shape[1])],
                            epochs=nb_epoch,
                            batch_size=batch_size,
                            shuffle=True,
                            validation_data=([X_val[:,i] for i in range(X_val.shape[1])], [X_val[:,i] for i in range(X_val.shape[1])]),
                            verbose=1,
                                callbacks= [earlystop] 
                            )
        
        # Create a plot of the model
        
        tf.keras.utils.plot_model(
            autoencoder,
            to_file="models/modelConcatamash{}.png".format(i),
            show_shapes=True,
            show_dtype=False,
            show_layer_names=False,
            rankdir="TB")
        
        # Save classifier for evaluation
        classifier = AutoEncoder.AutoEncoder(model=autoencoder)
        histstruct.add_classifier(histnames[i][0], classifier)        

In [14]:
train_concatamash_autoencoder(histstruct, histslist, vallist, autoencoders)


Now training model 1/12
Epoch 1/200


2022-06-21 22:39:42.852714: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-21 22:39:42.853182: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2900000000 Hz


21/21 [==============================] - 4s 56ms/step - loss: 0.0072 - dense_5_loss: 7.9005e-04 - dense_6_loss: 9.5852e-04 - dense_7_loss: 6.8858e-04 - dense_8_loss: 0.0012 - dense_9_loss: 8.2844e-04 - dense_10_loss: 9.5194e-04 - dense_11_loss: 8.6894e-04 - dense_12_loss: 9.8046e-04 - val_loss: 5.6638e-04 - val_dense_5_loss: 1.2112e-04 - val_dense_6_loss: 7.8187e-05 - val_dense_7_loss: 7.4354e-05 - val_dense_8_loss: 6.8538e-05 - val_dense_9_loss: 8.0874e-05 - val_dense_10_loss: 5.6441e-05 - val_dense_11_loss: 4.2490e-05 - val_dense_12_loss: 4.4382e-05
Epoch 2/200
21/21 [==============================] - 0s 14ms/step - loss: 2.1918e-04 - dense_5_loss: 3.9191e-05 - dense_6_loss: 2.9428e-05 - dense_7_loss: 2.7797e-05 - dense_8_loss: 3.3972e-05 - dense_9_loss: 2.5923e-05 - dense_10_loss: 2.1519e-05 - dense_11_loss: 2.0216e-05 - dense_12_loss: 2.1133e-05 - val_loss: 5.7226e-05 - val_dense_5_loss: 8.8653e-06 - val_dense_6_loss: 1.0641e-05 - val_dense_7_loss: 4.8776e-06 - val_dense_8_loss: 1.

21/21 [==============================] - 0s 14ms/step - loss: 1.0654e-05 - dense_5_loss: 5.4946e-07 - dense_6_loss: 3.4099e-06 - dense_7_loss: 5.3831e-07 - dense_8_loss: 1.9320e-06 - dense_9_loss: 8.5191e-07 - dense_10_loss: 1.3322e-06 - dense_11_loss: 8.8501e-07 - dense_12_loss: 1.1550e-06 - val_loss: 1.0184e-05 - val_dense_5_loss: 5.3474e-07 - val_dense_6_loss: 3.2107e-06 - val_dense_7_loss: 5.2812e-07 - val_dense_8_loss: 1.8307e-06 - val_dense_9_loss: 8.6045e-07 - val_dense_10_loss: 1.2573e-06 - val_dense_11_loss: 8.6095e-07 - val_dense_12_loss: 1.1012e-06
Epoch 16/200
21/21 [==============================] - 0s 13ms/step - loss: 1.0561e-05 - dense_5_loss: 5.6306e-07 - dense_6_loss: 3.3396e-06 - dense_7_loss: 5.5317e-07 - dense_8_loss: 1.9186e-06 - dense_9_loss: 8.6686e-07 - dense_10_loss: 1.2924e-06 - dense_11_loss: 8.9453e-07 - dense_12_loss: 1.1327e-06 - val_loss: 1.0257e-05 - val_dense_5_loss: 5.6989e-07 - val_dense_6_loss: 3.1866e-06 - val_dense_7_loss: 5.3997e-07 - val_dense_8

21/21 [==============================] - 0s 8ms/step - loss: 8.7467e-06 - dense_5_loss: 6.6251e-07 - dense_6_loss: 2.2838e-06 - dense_7_loss: 6.0961e-07 - dense_8_loss: 1.4027e-06 - dense_9_loss: 9.2315e-07 - dense_10_loss: 1.0336e-06 - dense_11_loss: 9.2658e-07 - dense_12_loss: 9.0482e-07 - val_loss: 9.2358e-06 - val_dense_5_loss: 6.5831e-07 - val_dense_6_loss: 2.5299e-06 - val_dense_7_loss: 5.9371e-07 - val_dense_8_loss: 1.5098e-06 - val_dense_9_loss: 9.2659e-07 - val_dense_10_loss: 1.0906e-06 - val_dense_11_loss: 9.1103e-07 - val_dense_12_loss: 1.0157e-06
Epoch 30/200
21/21 [==============================] - 0s 6ms/step - loss: 1.0492e-05 - dense_5_loss: 8.8538e-07 - dense_6_loss: 2.5834e-06 - dense_7_loss: 7.4738e-07 - dense_8_loss: 1.6998e-06 - dense_9_loss: 1.0712e-06 - dense_10_loss: 1.2658e-06 - dense_11_loss: 1.0611e-06 - dense_12_loss: 1.1781e-06 - val_loss: 9.2130e-06 - val_dense_5_loss: 7.5424e-07 - val_dense_6_loss: 2.3217e-06 - val_dense_7_loss: 6.3427e-07 - val_dense_8_l

...

Epoch 85/200
21/21 [==============================] - 0s 5ms/step - loss: 5.5563e-06 - dense_92_loss: 3.8869e-07 - dense_93_loss: 6.3983e-07 - dense_94_loss: 4.2318e-07 - dense_95_loss: 7.0241e-07 - dense_96_loss: 8.4692e-07 - dense_97_loss: 7.8539e-07 - dense_98_loss: 9.2303e-07 - dense_99_loss: 8.4689e-07 - val_loss: 5.7114e-06 - val_dense_92_loss: 3.4234e-07 - val_dense_93_loss: 6.8195e-07 - val_dense_94_loss: 3.9789e-07 - val_dense_95_loss: 7.8185e-07 - val_dense_96_loss: 8.2596e-07 - val_dense_97_loss: 8.4588e-07 - val_dense_98_loss: 9.2927e-07 - val_dense_99_loss: 9.0628e-07
Epoch 86/200
21/21 [==============================] - 0s 5ms/step - loss: 5.9165e-06 - dense_92_loss: 4.0425e-07 - dense_93_loss: 7.5623e-07 - dense_94_loss: 4.2983e-07 - dense_95_loss: 7.9100e-07 - dense_96_loss: 8.7055e-07 - dense_97_loss: 8.4137e-07 - dense_98_loss: 9.3601e-07 - dense_99_loss: 8.8730e-07 - val_loss: 1.1879e-05 - val_dense_92_loss: 6.7232e-07 - val_dense_93_loss: 2.1021e-06 - val_dense_94_l

Epoch 99/200
21/21 [==============================] - 0s 5ms/step - loss: 5.5129e-06 - dense_92_loss: 4.0409e-07 - dense_93_loss: 5.8990e-07 - dense_94_loss: 4.3911e-07 - dense_95_loss: 6.5150e-07 - dense_96_loss: 8.6429e-07 - dense_97_loss: 7.5924e-07 - dense_98_loss: 9.4476e-07 - dense_99_loss: 8.6007e-07 - val_loss: 5.2955e-06 - val_dense_92_loss: 3.7874e-07 - val_dense_93_loss: 4.2525e-07 - val_dense_94_loss: 4.5952e-07 - val_dense_95_loss: 5.3546e-07 - val_dense_96_loss: 8.7850e-07 - val_dense_97_loss: 6.7655e-07 - val_dense_98_loss: 1.1039e-06 - val_dense_99_loss: 8.3761e-07
Epoch 100/200
21/21 [==============================] - 0s 6ms/step - loss: 5.1688e-06 - dense_92_loss: 3.9155e-07 - dense_93_loss: 4.8604e-07 - dense_94_loss: 4.3203e-07 - dense_95_loss: 5.6450e-07 - dense_96_loss: 8.5186e-07 - dense_97_loss: 6.8547e-07 - dense_98_loss: 9.6704e-07 - dense_99_loss: 7.9032e-07 - val_loss: 7.8654e-06 - val_dense_92_loss: 5.3830e-07 - val_dense_93_loss: 1.2320e-06 - val_dense_94_

Epoch 113/200
21/21 [==============================] - 0s 6ms/step - loss: 5.9417e-06 - dense_92_loss: 4.6258e-07 - dense_93_loss: 6.7559e-07 - dense_94_loss: 4.8836e-07 - dense_95_loss: 6.8916e-07 - dense_96_loss: 9.5502e-07 - dense_97_loss: 7.9938e-07 - dense_98_loss: 9.8939e-07 - dense_99_loss: 8.8226e-07 - val_loss: 5.7707e-06 - val_dense_92_loss: 3.9808e-07 - val_dense_93_loss: 6.2846e-07 - val_dense_94_loss: 5.1430e-07 - val_dense_95_loss: 6.6999e-07 - val_dense_96_loss: 9.2238e-07 - val_dense_97_loss: 7.1056e-07 - val_dense_98_loss: 1.0017e-06 - val_dense_99_loss: 9.2519e-07
Epoch 114/200
21/21 [==============================] - 0s 6ms/step - loss: 5.0072e-06 - dense_92_loss: 4.0418e-07 - dense_93_loss: 4.7253e-07 - dense_94_loss: 4.4453e-07 - dense_95_loss: 4.8879e-07 - dense_96_loss: 8.5492e-07 - dense_97_loss: 6.6385e-07 - dense_98_loss: 9.4346e-07 - dense_99_loss: 7.3497e-07 - val_loss: 5.0844e-06 - val_dense_92_loss: 3.6202e-07 - val_dense_93_loss: 4.5202e-07 - val_dense_94

Epoch 127/200
21/21 [==============================] - 0s 6ms/step - loss: 4.3797e-06 - dense_92_loss: 3.6051e-07 - dense_93_loss: 3.4952e-07 - dense_94_loss: 4.0761e-07 - dense_95_loss: 3.5330e-07 - dense_96_loss: 7.9594e-07 - dense_97_loss: 5.8546e-07 - dense_98_loss: 8.7982e-07 - dense_99_loss: 6.4758e-07 - val_loss: 5.1020e-06 - val_dense_92_loss: 3.6728e-07 - val_dense_93_loss: 6.0986e-07 - val_dense_94_loss: 3.9556e-07 - val_dense_95_loss: 5.0466e-07 - val_dense_96_loss: 8.4779e-07 - val_dense_97_loss: 6.6480e-07 - val_dense_98_loss: 9.1497e-07 - val_dense_99_loss: 7.9711e-07
Epoch 128/200
21/21 [==============================] - 0s 5ms/step - loss: 4.7354e-06 - dense_92_loss: 3.8107e-07 - dense_93_loss: 4.7198e-07 - dense_94_loss: 4.1403e-07 - dense_95_loss: 4.6832e-07 - dense_96_loss: 7.9802e-07 - dense_97_loss: 6.2065e-07 - dense_98_loss: 8.8652e-07 - dense_99_loss: 6.9481e-07 - val_loss: 4.5325e-06 - val_dense_92_loss: 3.6010e-07 - val_dense_93_loss: 2.9664e-07 - val_dense_94

Epoch 141/200
21/21 [==============================] - 0s 6ms/step - loss: 4.2007e-06 - dense_92_loss: 3.6598e-07 - dense_93_loss: 2.7313e-07 - dense_94_loss: 4.0733e-07 - dense_95_loss: 2.8500e-07 - dense_96_loss: 8.1058e-07 - dense_97_loss: 5.5698e-07 - dense_98_loss: 9.0238e-07 - dense_99_loss: 5.9934e-07 - val_loss: 4.5000e-06 - val_dense_92_loss: 3.6254e-07 - val_dense_93_loss: 2.6236e-07 - val_dense_94_loss: 4.2482e-07 - val_dense_95_loss: 3.2480e-07 - val_dense_96_loss: 9.8387e-07 - val_dense_97_loss: 5.7699e-07 - val_dense_98_loss: 9.2536e-07 - val_dense_99_loss: 6.3923e-07
Epoch 142/200
21/21 [==============================] - 0s 6ms/step - loss: 4.2391e-06 - dense_92_loss: 3.5850e-07 - dense_93_loss: 2.9573e-07 - dense_94_loss: 4.0827e-07 - dense_95_loss: 3.1376e-07 - dense_96_loss: 8.1745e-07 - dense_97_loss: 5.5735e-07 - dense_98_loss: 8.8087e-07 - dense_99_loss: 6.0712e-07 - val_loss: 4.5952e-06 - val_dense_92_loss: 3.7986e-07 - val_dense_93_loss: 2.6846e-07 - val_dense_94

In [15]:
### Evaluate the Models for WP definition
def evaluate_models_train(histstruct):
    
    for histgroup in histnames:
        print('evaluating model for '+histgroup[0])
        print(histstruct.evaluate_classifier(histgroup)[0].shape)
    
    # get mse for training set
    if 'training' in histstruct.masks.keys(): masknames = ['dcson','highstat', 'training']
    else: masknames = ['dcson','highstat']
    mse_train = histstruct.get_scores_array( masknames=masknames )
    print('Found mse array for training set of following shape: {}'.format(mse_train.shape))
    
    # get mse for good set
    if 'good' in histstruct.masks.keys():
        mse_good = []
        for histname in histstruct.histnames:
            mse_good.append(histstruct.get_scores( histname=histname, masknames=['dcson','highstat','good'] ))
    else:
        mse_good = []
        for histname in histstruct.histnames:
            hists_good = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat']), 1000 )
            thismse = histstruct.classifiers[histname].evaluate( hists_good )
            mse_good.append( thismse )
            print(run)
    mse_good = np.array(mse_good)
    mse_good = np.transpose(mse_good)
    print('Found mse array for good set of following shape: {}'.format(mse_good.shape))
    
    # get mse for bad sets
    mse_bad = []
    for i in range(nbadruns):
        mse_bad.append( histstruct.get_scores_array( masknames=['dcson','highstat','bad{}'.format(i)] ) )
        print('Found mse array for bad set of following shape: {}'.format(mse_bad[i].shape))
        
    return [mse_train, mse_good, mse_bad]

In [16]:
(mse_train, mse_good_eval, mse_bad_eval) = evaluate_models_train(histstruct)

evaluating model for NormalizedHitResiduals_TIB__Layer__1
(2831,)
evaluating model for chargeInner_PXLayer_1
(2831,)
evaluating model for chargeInner_PXLayer_2
(2831,)
evaluating model for chargeInner_PXLayer_3
(2831,)
evaluating model for chargeInner_PXLayer_4
(2831,)
evaluating model for charge_PXDisk_+1
(2831,)
evaluating model for charge_PXDisk_-1
(2831,)
evaluating model for charge_PXDisk_+2
(2831,)
evaluating model for charge_PXDisk_-2
(2831,)
evaluating model for charge_PXDisk_+3
(2831,)
evaluating model for charge_PXDisk_-3
(2831,)
evaluating model for NormalizedHitResiduals_TOB__Layer__1
(2831,)
Found mse array for training set of following shape: (1678, 40)
Found mse array for good set of following shape: (1537, 40)
Found mse array for bad set of following shape: (44, 40)
Found mse array for bad set of following shape: (35, 40)
Found mse array for bad set of following shape: (46, 40)
Found mse array for bad set of following shape: (19, 40)
Found mse array for bad set of follo

In [17]:
### Plots and Distribution Analysis
def fit_mse_distribution(histstruct, mse_train):
    dimslist = []
    fitfunclist = []
    
    
    nhisttypes = len(histstruct.histnames)
    for i in range(0,nhisttypes-1):
        for j in range(i+1,nhisttypes):
            dimslist.append((i, j))
    
    plt.close('all')
    (npoints,ndims) = mse_train.shape
    
    
    # settings for GaussianKdeFitter
    scott_bw = npoints**(-1./(ndims+4))
    bw_method = 20*scott_bw
    # settings for HyperRectangleFitter
    quantiles = ([0.00062,0.0006,0.00015,0.00015,
                 0.0003,0.0003,0.00053,0.00065])
    
    
    for dims in dimslist:
        thismse = mse_train[:,dims]
        if training_mode=='global': 
            fitfunc = SeminormalFitter.SeminormalFitter(thismse)
            #fitfunc = HyperRectangleFitter.HyperRectangleFitter(thismse, 
            #                                                    [quantiles[dims[0]],quantiles[dims[1]]],
            #                                                    'up')
        else: fitfunc = GaussianKdeFitter.GaussianKdeFitter(thismse,bw_method=bw_method)
        #pu.plot_fit_2d(thismse, fitfunc=fitfunc, logprob=True, clipprob=True,
        #                onlycontour=False, xlims=30, ylims=30, 
        #                onlypositive=True, transparency=0.5,
        #                xaxtitle=histstruct.histnames[dims[0]], 
        #                yaxtitle=histstruct.histnames[dims[1]],
        #                title='density fit of lumisection MSE')
        ##plt.close('all') # release plot memory
        fitfunclist.append(fitfunc)
     
        
    if training_mode=='global': 
        fitfunc = SeminormalFitter.SeminormalFitter(mse_train)
        #fitfunc = HyperRectangleFitter.HyperRectangleFitter(mse_train, quantiles, 'up')
    else: fitfunc = GaussianKdeFitter.GaussianKdeFitter(mse_train,bw_method=bw_method)
    
    return fitfunc

In [18]:
fitfunc = fit_mse_distribution(histstruct, mse_train)

In [19]:
### Prepare MSEs for Working Point Definition
def mse_analysis(histstruct, mse_good_eval, mse_bad_eval, fitfunc):
    
    # Get the minimum log probability of histograms in good set
    print('--- good lumesections ---')
    logprob_good = np.log(fitfunc.pdf(mse_good_eval))
    print('length of log prob array: '+str(len(logprob_good)))
    print('minimum of log prob: '+str(np.min(logprob_good)))
    #print(sorted(logprob_good))
    
    print('--- bad lumisections ---')
    logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]
    #for lp in logprob_bad_parts: print(str(sorted(lp))+'\n\n')
    logprob_bad = np.concatenate(tuple(logprob_bad_parts))
    
    print('length of log prob array: '+str(len(logprob_bad)))
    print('maximum of log prob: '+str(np.max(logprob_bad)))
    #print(sorted(logprob_good))
    #print(sorted(logprob_bad))
    #print(logprob_bad)
    
    return [logprob_good, logprob_bad]

In [20]:
(logprob_good, logprob_bad) = mse_analysis(histstruct, mse_good_eval, mse_bad_eval, fitfunc)

--- good lumesections ---
length of log prob array: 1537
minimum of log prob: 370.82188385506277
--- bad lumisections ---
length of log prob array: 238
maximum of log prob: 410.0235904340274


/tmp/ipykernel_317647/688802665.py:12: RuntimeWarning: divide by zero encountered in log
  logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]


In [21]:
def evaluate_autoencoder_individual(histstruct):
    
    msewps = []
    for histname in histstruct.histnames:
        
        # Get histograms from histstruct
        X_test_good = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'good']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
        
        X_test_bad = X_test_bad = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'bad']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
        
        # Get each model from the histstruct
        autoencoder = histstruct.get_classifier(histname)
        
        # Getting evaluation criteria
        prediction_test_good = autoencoder.reconstruct(X_test_good)
        mse_test_good = aeu.mseTopNRaw(X_test_good, prediction_test_good, n=10 )
        prediction_test_bad = autoencoder.reconstruct(X_test_bad)
        mse_test_bad = aeu.mseTopNRaw(X_test_bad, prediction_test_bad, n=10 )
        
        if userfriendly:
            print('Average MSE on good set: ' + str(np.mean(mse_test_good)))
            print('Average MSE on bad set: ' + str(np.mean(mse_test_bad)))
        
        if createPlots:
            # Number of plots of each type to generate per model (so nplot * 2 * len(model))
            nplot = 3
            
            # Good examples
            print('Examples of good histograms and reconstruction:')
            randint = np.random.choice(np.arange(len(X_test_good)),size=nplot,replace=False)
            for i in randint: 
                histlist = [X_test_good[int(i),:],prediction_test_good[int(i),:]]
                labellist = ['data','reconstruction']
                colorlist = ['black','blue']
                pu.plot_hists(histlist,colorlist=colorlist,labellist=labellist)
                plt.show()
            
            # Bad examples
            print('Examples of bad histograms and reconstruction:')
            randint = np.random.choice(np.arange(len(X_test_bad)),size=nplot,replace=False)
            for i in randint:
                histlist = [X_test_bad[int(i),:],prediction_test_bad[int(i),:]]
                labellist = ['data','reconstruction']
                colorlist = ['black','blue']
                pu.plot_hists(histlist,colorlist=colorlist,labellist=labellist)
                plt.show()
        
        # Attaching the bad histograms as a new set of rows under the good histograms
        validation_data = np.vstack((X_test_good, X_test_bad))
        validation_preds = np.vstack((prediction_test_good, prediction_test_bad))
        # Creating labels to differentiate the data when we go to compare predictions
        #     with actual label
        labels = np.hstack((np.zeros(len(X_test_good)), np.ones(len(X_test_bad))))
        
        # Pick a working point to see 
        msewp = 0.5*(np.mean(mse_test_bad) - np.mean(mse_test_good))
        print("Selected working point: " + str(msewp))
        
        # Get data to pick a good working point for future evaluation
        scores = aeu.mseTop10Raw(validation_data, validation_preds)
        nsig = np.sum(labels)
        nback = np.sum(1-labels)
        
        # Get some metrics for the user
        tp = np.sum(np.where((labels==1) & (scores>msewp),1,0))/nsig
        fp = np.sum(np.where((labels==0) & (scores>msewp),1,0))/nback
        tn = 1-fp
        fn = 1-tp
        
        accuracy = (tp + tn) / (tp + fp + tn + fn)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = (2*precision*recall) / (precision + recall)
        
        if userfriendly:
            print(accuracy)
            print(precision)
            print(recall)
            print(f1)
        
    return msewps

In [22]:
def evaluate_autoencoders_combined(logprob_good, logprob_bad, biasFactor):
    labels_good = np.zeros(len(logprob_good)) # background: label = 0
    labels_bad = np.ones(len(logprob_bad)) # signal: label = 1
    
    badMin = min(np.where(logprob_bad != -np.inf, logprob_bad, np.inf))
    goodMax = max(np.where(logprob_good != np.inf, logprob_good, -np.inf))
    
    # Getting rid of infinities
    logprob_good[logprob_good > 500] = goodMax
    logprob_bad[logprob_bad < 0] = badMin
    # These only take effect if a histogram is grossly misclassified
    logprob_good[logprob_good < badMin] = badMin
    logprob_bad[logprob_bad > goodMax] = goodMax
    
    labels = np.concatenate(tuple([labels_good,labels_bad]))
    scores = np.concatenate(tuple([-logprob_good,-logprob_bad]))
    scores = aeu.clip_scores( scores )
    
    pu.plot_score_dist(scores, labels, siglabel='anomalous', sigcolor='r', 
                       bcklabel='good', bckcolor='g', 
                       nbins=200, normalize=True,
                       xaxtitle='negative logarithmic probability',
                       yaxtitle='number of lumisections (normalized)')
      
    # Plot ROC curve for analysis
    auc = aeu.get_roc(scores, labels, mode='geom', doprint=False)
    
    # Setting a threshold, below this working point defines anomalous data
    # Average is biased towards better recall per user specifications
    logprob_threshold = (1/(biasFactor + 1)) * (biasFactor*np.mean(logprob_good) + np.mean(logprob_bad))
    # Or set manual
    # logprob_threshold = 424
    (_, _, _, tp, fp, tn, fn) = aeu.get_confusion_matrix(scores,labels,-logprob_threshold)
    print('Selected logprob threshold of ' + str(logprob_threshold))
    
    # Get metrics for analysis
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_measure = (1 + biasFactor * biasFactor) * ((precision * recall) / ((biasFactor * biasFactor * precision) + recall)) 
    
    print('Accuracy: ' + str(accuracy))
    print('Precision: ' + str(precision))
    print('Recall: ' + str(recall))
    print('F-Measure: ' + str(f_measure))
    
    return logprob_threshold

In [23]:
logprob_threshold = evaluate_autoencoders_combined(logprob_good, logprob_bad)

TypeError: evaluate_autoencoders_combined() missing 1 required positional argument: 'biasFactor'

In [ ]:
msewps = evaluate_autoencoder_individual(histstruct)